## Using Python to Integrate MongoDB Data into an ETL Process
Modern Data Warehousing and Analytics solutions frequently use languages like Python or Scala to extract data from numerous sources, including relational database management systems, NoSQL database systems, real-time streaming endpoints and Data Lakes. These languages can then be used to perform many types of transformation before then loading the data into a variety of destinations including file systems and data warehouses. This data can then be consumed by data scientists or business analysts.

In this lab you will build upon the **Northwind_DW2** dimensional database from Lab 3; however, you will be integrating new data sourced from an instance of MongoDB. The new data will be concerned with new business processes; inventory and purchasing. You will continue to interact with both the source systems (MongoDB and MySQL), and the destination system (the Northwind_DW2 data warehouse) from a remote client running Python (Jupyter Notebooks). 

Just as in Lab 3, you will fetch data into Pandas DataFrames, perform all the necessary transformations in-memory on the client, and then push the newly transformed DataFrame to the RDBMS data warehouse using a Pandas function that will create the table and fill it with data with a single operation.

### Prerequisites:
This notebook uses the PyMongo database connectivity library to connect to MySQL databases; therefore, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install pymongo[srv]`

#### Import the Necessary Libraries

In [62]:
%pip install "pymongo[srv]"


Note: you may need to restart the kernel to use updated packages.


In [63]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [64]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.34
Running PyMongo Version: 4.15.3


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [65]:
mysql_args = {
    "uid" : "root",
    "pwd" : "eleanorfox",
    "hostname" : "localhost",
    "dbname" : "northwind_dw2"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "elenaorfox",
    "password" : "eleanorfox",
    "cluster_name" : "eleanorfox",
    "cluster_subnet" : "q5kyufx",
    "cluster_location" : "atlas", # "local"
    "db_name" : "northwind_purchasing"
}

#### Define Functions for Getting Data From and Setting Data Into Databases

In [66]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Populate MongoDB with Source Data
You only need to run this cell once; however, the operation is *idempotent*.  In other words, it can be run multiple times without changing the end result.

In [68]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"suppliers" : 'northwind_suppliers.json',
              "invoices" : 'northwind_invoices.json',
              "purchase_orders" : 'northwind_purchase_orders.json',
              "inventory_transactions" : 'northwind_inventory_transactions.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         

### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [69]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "invoices"

df_invoices = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_invoices.head(2)

,id,order_id,invoice_date,due_date,tax,shipping,amount_due
0,5,31,2006-03-22 16:08:59,None,0.0,0.0,0.0
1,6,32,2006-03-22 16:10:27,None,0.0,0.0,0.0


In [70]:
# TODO: Extract data from the "Suppliers" collection 

client = get_mongo_client (**mongodb_args) 

query ={}
collection = "suppliers"

df_suppliers = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_suppliers.head(2)

,id,company,last_name,first_name,job_title
0,1,Supplier A,Andersen,Elizabeth A.,Sales Manager
1,2,Supplier B,Weiler,Cornelia,Sales Manager


#### 1.2. Lookup the Invoice Date Keys from the Date Dimension Table.
Here we see an example of where a dimension cross-references another dimension; the Date dimension.  The Date dimension is a classic example of a **Role-Playing dimension**. Dates in-and-of themselves are universal; however, when applied to specific events they take on the identity of those events. Here, the *dim_date* table takes on the identity of *dim_invoice_date* to supply invoice date keys to the *dim_invoices* table.

##### 1.2.1. Get the Data from the Date Dimension Table.
First, fetch the Surrogate Primary Key (date_key) and the Business Key (full_date) from the Date Dimension table using the **get_sql_dataframe()** function. Be certain to cast the **full_date** column to the **datetime64[ns]** data type using the **.astype()** function that is native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [71]:
sql_dim_date = "SELECT date_key, full_date FROM northwind_dw2.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


##### 1.2.2. Lookup the Surrogate Primary Key (date_key) that Corresponds to the invoice_date Column

In [72]:
df_dim_invoice_date = df_dim_date.rename(columns={"date_key" : "invoice_date_key", "full_date" : "invoice_date"})
df_invoices.invoice_date = df_invoices.invoice_date.astype('datetime64[ns]').dt.date
df_invoices = pd.merge(df_invoices, df_dim_invoice_date, on='invoice_date', how='left')
df_invoices.drop(['invoice_date'], axis=1, inplace=True)
df_invoices.head(2)

,id,order_id,due_date,tax,shipping,amount_due,invoice_date_key
0,5,31,None,0.0,0.0,0.0,20060322
1,6,32,None,0.0,0.0,0.0,20060322


#### 1.3. Perform Any Necessary Transformations to the DataFrames

In [73]:
# 1. Rename the "id" column to reflect the entity as it will serve as the business key for lookup operations
df_invoices.rename(columns={"id":"invoice_id"}, inplace=True)

# 2. Since there are no values in the 'due_date' column go ahead and drop it.
df_invoices.drop(['due_date'], axis=1, inplace=True)

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_invoices.insert(0, "invoice_key", range(1, df_invoices.shape[0]+1))
df_invoices.head(2)

,invoice_key,invoice_id,order_id,tax,shipping,amount_due,invoice_date_key
0,1,5,31,0.0,0.0,0.0,20060322
1,2,6,32,0.0,0.0,0.0,20060322


In [74]:
# 1. TODO: Rename the "id" column to reflect the entity as it will serve as the business key for lookup operations
df_suppliers.rename(columns={"id":"suppliers_id"}, inplace=True)

# 2. TODO: Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_suppliers.insert(0, "suppliers_key", range(1, df_suppliers.shape[0]+1))
df_suppliers.head(2)

,suppliers_key,suppliers_id,company,last_name,first_name,job_title
0,1,1,Supplier A,Andersen,Elizabeth A.,Sales Manager
1,2,2,Supplier B,Weiler,Cornelia,Sales Manager


#### 1.3. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

Here we will call our **set_dataframe( )** function to create each dimension table. This function expects a number of parameters including the usual connection information (e.g., user_id, password, MySQL server name and database), the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, the *name* of the column we wish to designate as the *primary_key* column, and finally, the database operation (insert or update). 

In [75]:
dataframe = df_invoices
table_name = 'dim_invoices'
primary_key = 'invoice_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [76]:
# TODO: Upload the "Suppliers" dataframe to create the new "dim_suppliers" dimension table

dataframe = df_suppliers
table_name = 'dim_suppliers'
primary_key = 'suppliers_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### 1.4. Validate that the New Dimension Tables were Created.

In [77]:
sql_invoices = "SELECT * FROM northwind_dw2.dim_invoices;"
df_dim_invoices = get_sql_dataframe(sql_invoices, **mysql_args)
df_dim_invoices.head(2)

,invoice_key,invoice_id,order_id,tax,shipping,amount_due,invoice_date_key
0,1,5,31,0.0,0.0,0.0,20060322
1,2,6,32,0.0,0.0,0.0,20060322


In [78]:
# TODO: Validate the new "dim_suppliers" table in the northwind_dw2 data warehouse.
sql_suppliers = "SELECT * FROM northwind_dw2.dim_suppliers;"
df_dim_suppliers = get_sql_dataframe(sql_suppliers, **mysql_args)
df_dim_suppliers.head(2)

,suppliers_key,suppliers_id,company,last_name,first_name,job_title
0,1,1,Supplier A,Andersen,Elizabeth A.,Sales Manager
1,2,2,Supplier B,Weiler,Cornelia,Sales Manager


### 2.0. Create and Populate the New Fact Tables
#### 2.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [79]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "purchase_orders"

df_fact_pos = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_fact_pos.head(2)

,purchase_order_id,supplier_id,po_detail_product_id,po_detail_inventory_id,created_by,submitted_date,creation_date,po_detail_date_received,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,approved_by,approved_date,submitted_by
0,90,1,1,59.0,2.0,2006-01-14 00:00:00,2006-01-22 00:00:00,2006-01-22 00:00:00,Approved,1,14.0,40.0,0.0,0.0,None,0.0,2.0,2006-01-22 00:00:00,2
1,90,1,34,60.0,2.0,2006-01-14 00:00:00,2006-01-22 00:00:00,2006-01-22 00:00:00,Approved,1,10.0,60.0,0.0,0.0,None,0.0,2.0,2006-01-22 00:00:00,2


In [80]:
# TODO: Extract data for your new "Inventory Transactions" Fact Table
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "inventory_transactions"

df_invent_trans = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_invent_trans.head(2)

,inventory_transaction_id,inventory_transaction_type,transaction_created_date,transaction_modified_date,product_id,quantity,purchase_order_id,customer_order_id
0,35,Purchased,2006-03-22 16:02:28,2006-03-22 16:02:28,80,75,None,None
1,36,Purchased,2006-03-22 16:02:48,2006-03-22 16:02:48,72,40,None,None


#### 2.2. Lookup the DateKeys from the Date Dimension Table.
**2.2.1.** For each date typed column in the **purchase orders** fact table, lookup the corresponding Surrogate Primary Key column. Be certain to cast the date typed column to the **datetime64[ns]** data type using the **.astype()** function that is native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [81]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "submitted_date" Column.
df_dim_submitted_date = df_dim_date.rename(columns={"date_key" : "submitted_date_key", "full_date" : "submitted_date"})
df_fact_pos.submitted_date = df_fact_pos.submitted_date.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_dim_submitted_date, on='submitted_date', how='left')
df_fact_pos.drop(['submitted_date'], axis=1, inplace=True)
df_fact_pos.head(2)

,purchase_order_id,supplier_id,po_detail_product_id,po_detail_inventory_id,created_by,creation_date,po_detail_date_received,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,approved_by,approved_date,submitted_by,submitted_date_key
0,90,1,1,59.0,2.0,2006-01-22 00:00:00,2006-01-22 00:00:00,Approved,1,14.0,40.0,0.0,0.0,None,0.0,2.0,2006-01-22 00:00:00,2,20060114
1,90,1,34,60.0,2.0,2006-01-22 00:00:00,2006-01-22 00:00:00,Approved,1,10.0,60.0,0.0,0.0,None,0.0,2.0,2006-01-22 00:00:00,2,20060114


In [82]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "creation_date" Column.
# need to fix
df_dim_creation_date = df_dim_date.rename(columns={"date_key" : "creation_date_key", "full_date" : "creation_date"})
df_fact_pos.creation_date = df_fact_pos.creation_date.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_dim_creation_date, on='creation_date', how='left')
df_fact_pos.drop(['creation_date'], axis=1, inplace=True)
df_fact_pos.head(2)

,purchase_order_id,supplier_id,po_detail_product_id,po_detail_inventory_id,created_by,po_detail_date_received,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,approved_by,approved_date,submitted_by,submitted_date_key,creation_date_key
0,90,1,1,59.0,2.0,2006-01-22 00:00:00,Approved,1,14.0,40.0,0.0,0.0,None,0.0,2.0,2006-01-22 00:00:00,2,20060114,20060122
1,90,1,34,60.0,2.0,2006-01-22 00:00:00,Approved,1,10.0,60.0,0.0,0.0,None,0.0,2.0,2006-01-22 00:00:00,2,20060114,20060122


In [83]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "po_detail_date_received" Column.
# need to fix
df_dim_po_detail_date_received = df_dim_date.rename(columns={"date_key" : "po_detail_date_received_key", "full_date" : "po_detail_date_received"})
df_fact_pos.po_detail_date_received = df_fact_pos.po_detail_date_received.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_dim_po_detail_date_received, on='po_detail_date_received', how='left')
df_fact_pos.drop(['po_detail_date_received'], axis=1, inplace=True)
df_fact_pos.head(2)



,purchase_order_id,supplier_id,po_detail_product_id,po_detail_inventory_id,created_by,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,approved_by,approved_date,submitted_by,submitted_date_key,creation_date_key,po_detail_date_received_key
0,90,1,1,59.0,2.0,Approved,1,14.0,40.0,0.0,0.0,None,0.0,2.0,2006-01-22 00:00:00,2,20060114,20060122,20060122.0
1,90,1,34,60.0,2.0,Approved,1,10.0,60.0,0.0,0.0,None,0.0,2.0,2006-01-22 00:00:00,2,20060114,20060122,20060122.0


In [84]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "approved_date" Column.
# need to fix
df_dim_approved_date = df_dim_date.rename(columns={"date_key" : "approved_date_key", "full_date" : "approved_date"})
df_fact_pos.approved_date = df_fact_pos.approved_date.astype('datetime64[ns]').dt.date
df_fact_pos = pd.merge(df_fact_pos, df_dim_approved_date, on='approved_date', how='left')
df_fact_pos.drop(['approved_date'], axis=1, inplace=True)
df_fact_pos.head(2)

,purchase_order_id,supplier_id,po_detail_product_id,po_detail_inventory_id,created_by,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,approved_by,submitted_by,submitted_date_key,creation_date_key,po_detail_date_received_key,approved_date_key
0,90,1,1,59.0,2.0,Approved,1,14.0,40.0,0.0,0.0,None,0.0,2.0,2,20060114,20060122,20060122.0,20060122.0
1,90,1,34,60.0,2.0,Approved,1,10.0,60.0,0.0,0.0,None,0.0,2.0,2,20060114,20060122,20060122.0,20060122.0


**2.2.2.** Then, for each date typed column in the **inventory transaction** fact table, lookup the corresponding Surrogate Primary Key column. Be certain to cast the date typed column to the **datetime64[ns]** data type using the **.astype()** function, and extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [85]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "transaction_created_date" Column.
df_dim_tran_created_date = df_dim_date.rename(columns={"date_key" : "transaction_created_date_key", "full_date" : "transaction_created_date"})
df_invent_trans.transaction_created_date = df_invent_trans.transaction_created_date.astype('datetime64[ns]').dt.date
df_invent_trans = pd.merge(df_invent_trans, df_dim_tran_created_date, on='transaction_created_date', how='left')    
df_invent_trans.drop(['transaction_created_date'], axis=1, inplace=True)
df_invent_trans.head(2)



,inventory_transaction_id,inventory_transaction_type,transaction_modified_date,product_id,quantity,purchase_order_id,customer_order_id,transaction_created_date_key
0,35,Purchased,2006-03-22 16:02:28,80,75,None,None,20060322
1,36,Purchased,2006-03-22 16:02:48,72,40,None,None,20060322


In [86]:
# TODO: Lookup the Primary Key (date_key) that Corresponds to the "transaction_modified_date" Column.
df_dim_tran_modified_date = df_dim_date.rename(columns={"date_key" : "transaction_modified_date_key", "full_date" : "transaction_modified_date"})
df_invent_trans.transaction_modified_date = df_invent_trans.transaction_modified_date.astype('datetime64[ns]').dt.date
df_invent_trans = pd.merge(df_invent_trans, df_dim_tran_modified_date, on='transaction_modified_date', how='left')    
df_invent_trans.drop(['transaction_modified_date'], axis=1, inplace=True)
df_invent_trans.head(2)

,inventory_transaction_id,inventory_transaction_type,product_id,quantity,purchase_order_id,customer_order_id,transaction_created_date_key,transaction_modified_date_key
0,35,Purchased,80,75,None,None,20060322,20060322
1,36,Purchased,72,40,None,None,20060322,20060322


#### 2.3. Lookup the Primary Keys from the Dimension Tables
**Foreign key relationships** must be established between each newly-crafted **Fact table** and each related **Dimension table**.

##### 2.3.1. First, fetch the Surrogate Primary Key and the Business Key from each Dimension table.

In [87]:
sql_dim_invoices = "SELECT invoice_key, invoice_id FROM northwind_dw2.dim_invoices;"
df_dim_invoices = get_sql_dataframe(sql_dim_invoices, **mysql_args)
df_dim_invoices.head(2)

,invoice_key,invoice_id
0,1,5
1,2,6


In [88]:
# TODO: Extract the 'primary key' and the 'business key' from your new "dim_suppliers" dimension table
sql_dim_suppliers = "SELECT suppliers_key, suppliers_id FROM northwind_dw2.dim_suppliers;"
df_dim_suppliers = get_sql_dataframe(sql_dim_suppliers, **mysql_args)
df_dim_suppliers.head(2)

,suppliers_key,suppliers_id
0,1,1
1,2,2


In [89]:
# TODO: Extract the 'primary key' and the 'business key' from the "dim_employees" dimension table
sql_dim_employees = "SELECT employee_key, employee_id FROM northwind_dw2.dim_employees;"
df_dim_employees = get_sql_dataframe(sql_dim_employees, **mysql_args)
df_dim_employees.head(2)

,employee_key,employee_id
0,1,1
1,2,2


In [90]:
# TODO: Extract the 'primary key' and the 'business key' from the "dim_products" dimension table
sql_dim_products = "SELECT product_key, product_id FROM northwind_dw2.dim_products;"
df_dim_products = get_sql_dataframe(sql_dim_products, **mysql_args)
df_dim_products.head(2)

,product_key,product_id
0,1,1
1,2,3


##### 2.3.2. Next, use Business Keys to lookup corresponding Surrogate Primary Keys in the Dimension tables
**2.3.2.1. Inventory Fact table.**  The inventory fact table contains a reference to the *dim_products* table by way of the *product_id* column that enables us to lookup the corresponding *product_key*.

In [91]:
# TODO: Merge the "fact_inventory" and "dim_products" dataframes on the 'product_id' to
# get the 'product_key'. Then drop the 'product_id' column and display the results.
df_fact_inventory = df_invent_trans
df_fact_inventory = pd.merge(df_fact_inventory, df_dim_products, on='product_id', how='inner')
df_fact_inventory.drop(['product_id'], axis=1, inplace=True)
df_fact_inventory.head(2)

,inventory_transaction_id,inventory_transaction_type,quantity,purchase_order_id,customer_order_id,transaction_created_date_key,transaction_modified_date_key,product_key
0,35,Purchased,75,None,None,20060322,20060322,27
1,36,Purchased,40,None,None,20060322,20060322,24


In [92]:
# TODO: Insert a new 'fact_inventory_transaction_key' column, with an ever-incrementing
# numeric value, to serve as the surrogate primary key. Then display the results.
df_fact_inventory.insert(0, "fact_inventory_transaction_key", range(1, df_fact_inventory.shape[0]+1))
df_fact_inventory.head(2)

,fact_inventory_transaction_key,inventory_transaction_id,inventory_transaction_type,quantity,purchase_order_id,customer_order_id,transaction_created_date_key,transaction_modified_date_key,product_key
0,1,35,Purchased,75,None,None,20060322,20060322,27
1,2,36,Purchased,40,None,None,20060322,20060322,24


**2.3.2.2. Purchase Order Fact table**. Here we see another example of a **role-playing dimension**; the *Employee* dimension. The employee dimension is another classic example of a Role-Playing dimension. In-and-of themselves Employees are generic; however, when applied to specific roles they take on the identity of those positions (roles). Here, the *dim_employee* table takes on the identities of *creator, approver, and submitter* in supplying employee keys to the *dim_fact_pos* table.

We also see an example of where one fact table is referenced by another fact table as if it were a dimension table; i.e., a **fact dimension.** In this instance the purchase orders table *fact_pos* contains a reference to the inventory transaction table *fact_inventory* by way of the *inventory_transaction_id* column, enabling us to lookup the corresponding *fact_inventory_key*.

In [93]:
# Lookup the Surrogate Primary Key (employee_key) that Corresponds to the "created_by" Column.
df_dim_creator = df_dim_employees.rename(columns={"employee_key":"created_by_key", "employee_id":"created_by"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_creator, on='created_by', how='inner')
df_fact_pos.drop(['created_by'], axis=1, inplace=True)
df_fact_pos.head(2)

,purchase_order_id,supplier_id,po_detail_product_id,po_detail_inventory_id,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,approved_by,submitted_by,submitted_date_key,creation_date_key,po_detail_date_received_key,approved_date_key,created_by_key
0,90,1,1,59.0,Approved,1,14.0,40.0,0.0,0.0,None,0.0,2.0,2,20060114,20060122,20060122.0,20060122.0,2
1,90,1,34,60.0,Approved,1,10.0,60.0,0.0,0.0,None,0.0,2.0,2,20060114,20060122,20060122.0,20060122.0,2


In [94]:
# TODO: Lookup the Surrogate Primary Key (employee_key) that Corresponds to the "approved_by" Column.
df_dim_approver = df_dim_employees.rename(columns={"employee_key":"approved_by_key", "employee_id":"approved_by"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_approver, on='approved_by', how='inner')
df_fact_pos.drop(['approved_by'], axis=1, inplace=True)
df_fact_pos.head(2)

,purchase_order_id,supplier_id,po_detail_product_id,po_detail_inventory_id,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,submitted_by,submitted_date_key,creation_date_key,po_detail_date_received_key,approved_date_key,created_by_key,approved_by_key
0,90,1,1,59.0,Approved,1,14.0,40.0,0.0,0.0,None,0.0,2,20060114,20060122,20060122.0,20060122.0,2,2
1,90,1,34,60.0,Approved,1,10.0,60.0,0.0,0.0,None,0.0,2,20060114,20060122,20060122.0,20060122.0,2,2


In [95]:
# TODO: Lookup the Surrogate Primary Key (employee_key) that Corresponds to the "submitted_by" Column.
df_dim_submitter = df_dim_employees.rename(columns={"employee_key":"submitted_by_key", "employee_id":"submitted_by"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_submitter, on='submitted_by', how='inner')
df_fact_pos.drop(['submitted_by'], axis=1, inplace=True)
df_fact_pos.head(2)

,purchase_order_id,supplier_id,po_detail_product_id,po_detail_inventory_id,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,submitted_date_key,creation_date_key,po_detail_date_received_key,approved_date_key,created_by_key,approved_by_key,submitted_by_key
0,90,1,1,59.0,Approved,1,14.0,40.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2
1,90,1,34,60.0,Approved,1,10.0,60.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2


Since the *df_fact_pos* fact table contains references to suppliers and products, lookup the surrogate primary keys from the *dim_suppliers* and *dim_products* tables that correspond to the *supplier_id* and *product_id* columns.

In [96]:
# TODO: Lookup the Surrogate Primary Key (supplier_key) that Corresponds to the "supplier_id" Column.
df_dim_suppliers = df_dim_suppliers.rename(columns={"suppliers_key":"supplier_key", "suppliers_id":"supplier_id"})
df_fact_pos = pd.merge(df_fact_pos, df_dim_suppliers, on='supplier_id', how='inner')
df_fact_pos.drop(['supplier_id'], axis=1, inplace=True)
df_fact_pos.head(2)





,purchase_order_id,po_detail_product_id,po_detail_inventory_id,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,submitted_date_key,creation_date_key,po_detail_date_received_key,approved_date_key,created_by_key,approved_by_key,submitted_by_key,supplier_key
0,90,1,59.0,Approved,1,14.0,40.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2,1
1,90,34,60.0,Approved,1,10.0,60.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2,1


In [ ]:
# TODO: Lookup the Primary Key (product_key) that Corresponds to the "po_detail_product_id" Column.
# Remember that you'll first have to rename it to "product_id" in the "df_fact_pos" dataframe.


# 1) Rename the fact column first
df_fact_pos.rename(columns={"po_detail_product_id": "product_id"}, inplace=True)
df_fact_pos = pd.merge(df_fact_pos, df_dim_products[["product_id", "product_key"]], on="product_id", how="inner")
df_fact_pos.drop(columns=["product_id"], inplace=True)
df_fact_pos.head(2)


,purchase_order_id,po_detail_inventory_id,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,submitted_date_key,creation_date_key,po_detail_date_received_key,approved_date_key,created_by_key,approved_by_key,submitted_by_key,supplier_key,product_key
0,90,59.0,Approved,1,14.0,40.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2,1,1
1,90,60.0,Approved,1,10.0,60.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2,1,13


In [98]:
# TODO: Insert a new "fact_purchase_order_key" column, with an ever-incrementing 
# numeric value, to serve as the surrogate primary key. Then display the results.
df_fact_pos.insert(0, "fact_purchase_order_key", range(1, df_fact_pos.shape[0]+1))
df_fact_pos.head(2)

,fact_purchase_order_key,purchase_order_id,po_detail_inventory_id,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,submitted_date_key,creation_date_key,po_detail_date_received_key,approved_date_key,created_by_key,approved_by_key,submitted_by_key,supplier_key,product_key
0,1,90,59.0,Approved,1,14.0,40.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2,1,1
1,2,90,60.0,Approved,1,10.0,60.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2,1,13


Since the purchase orders table *fact_pos* contains a reference to the inventory transaction table *fact_inventory* by way of the *inventory_transaction_id* column, we can lookup the corresponding *fact_inventory_transaction_key*. In this case, the *fact_inventory* table plays the role of a dimension table. This is called a **fact dimension**.

In [99]:
# The inventory fact table acts as a Fact Dimension: po_detail_inventory_id
df_dim_fact_inventory = df_fact_inventory[['fact_inventory_transaction_key','inventory_transaction_id']]
df_fact_pos.rename(columns={"po_detail_inventory_id":"inventory_transaction_id"}, inplace=True)
df_fact_pos = pd.merge(df_fact_pos, df_dim_fact_inventory, on='inventory_transaction_id', how='left')
df_fact_pos.drop(['inventory_transaction_id'], axis=1, inplace=True)
df_fact_pos.head(2)

,fact_purchase_order_key,purchase_order_id,purchase_order_status,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount,submitted_date_key,creation_date_key,po_detail_date_received_key,approved_date_key,created_by_key,approved_by_key,submitted_by_key,supplier_key,product_key,fact_inventory_transaction_key
0,1,90,Approved,1,14.0,40.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2,1,1,25.0
1,2,90,Approved,1,10.0,60.0,0.0,0.0,None,0.0,20060114,20060122,20060122.0,20060122.0,2,2,2,1,13,26.0


#### 2.4. Perform Any Necessary Transformations to the DataFrames

In [100]:
# Rename the Foreign Key Columns
column_name_map = {"transaction_created_date_key" : "created_date_key",
                   "transaction_modified_date_key" : "modified_date_key",
                  }

df_fact_inventory.rename(columns=column_name_map, inplace=True)

# Reorder the Columns
ordered_columns = ['fact_inventory_transaction_key','inventory_transaction_id'
                   ,'inventory_transaction_type','product_key'
                   ,'created_date_key','modified_date_key','quantity']

df_fact_inventory = df_fact_inventory[ordered_columns]
df_fact_inventory.head(2)

,fact_inventory_transaction_key,inventory_transaction_id,inventory_transaction_type,product_key,created_date_key,modified_date_key,quantity
0,1,35,Purchased,27,20060322,20060322,75
1,2,36,Purchased,24,20060322,20060322,40


In [101]:
# Change the 'po_detail_posted_to_inventory' column values from 1 and 0 to 'True' and 'False'.
df_fact_pos['posted_to_inventory'] = df_fact_pos.po_detail_posted_to_inventory.map({1 :'True', 0 :'False'})
df_fact_pos.drop('po_detail_posted_to_inventory', axis=1, inplace=True)
df_fact_pos.rename(columns={'posted_to_inventory' : 'po_detail_posted_to_inventory'}, inplace=True)

# Reorder the Columns
reordered_columns = ['fact_purchase_order_key','purchase_order_id','supplier_key','product_key'
                     ,'fact_inventory_transaction_key','created_by_key','creation_date_key'
                     ,'submitted_by_key','submitted_date_key','approved_by_key','approved_date_key'
                     ,'purchase_order_status','po_detail_date_received_key'
                     ,'po_detail_posted_to_inventory','po_detail_unit_cost','po_detail_quantity'
                     ,'shipping_fee','taxes','payment_date','payment_amount']

df_fact_pos = df_fact_pos[reordered_columns]
df_fact_pos.head(2)

,fact_purchase_order_key,purchase_order_id,supplier_key,product_key,fact_inventory_transaction_key,created_by_key,creation_date_key,submitted_by_key,submitted_date_key,approved_by_key,approved_date_key,purchase_order_status,po_detail_date_received_key,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount
0,1,90,1,1,25.0,2,20060122,2,20060114,2,20060122.0,Approved,20060122.0,True,14.0,40.0,0.0,0.0,None,0.0
1,2,90,1,13,26.0,2,20060122,2,20060114,2,20060122.0,Approved,20060122.0,True,10.0,60.0,0.0,0.0,None,0.0


#### 2.5. Load Newly Transformed MongoDB Data into the Northwind_DW2 Data Warehouse

In [102]:
dataframe = df_fact_pos
table_name = 'fact_purchase_orders'
primary_key = 'fact_purchase_order_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [103]:
# TODO: Upload the "Inventory Transaction" dataframe to create the new "fact_inventory_transactions" fact table. 

dataframe = df_fact_inventory
table_name = 'fact_inventory_transactions'
primary_key = 'fact_inventory_transaction_key'
db_operation = "insert"
set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### 2.6. Validate that the New Fact Tables were Created

In [104]:
# TODO: Validate the correctness of the new "Purchase Orders" fact table.
sql_purchase_orders = "SELECT * FROM northwind_dw2.fact_purchase_orders;"
df_fact_inventory_transaction = get_sql_dataframe(sql_purchase_orders, **mysql_args)
df_fact_inventory_transaction.head(2)

,fact_purchase_order_key,purchase_order_id,supplier_key,product_key,fact_inventory_transaction_key,created_by_key,creation_date_key,submitted_by_key,submitted_date_key,approved_by_key,approved_date_key,purchase_order_status,po_detail_date_received_key,po_detail_posted_to_inventory,po_detail_unit_cost,po_detail_quantity,shipping_fee,taxes,payment_date,payment_amount
0,1,90,1,1,25.0,2,20060122,2,20060114,2,20060122.0,Approved,20060122.0,True,14.0,40.0,0.0,0.0,None,0.0
1,2,90,1,13,26.0,2,20060122,2,20060114,2,20060122.0,Approved,20060122.0,True,10.0,60.0,0.0,0.0,None,0.0


In [105]:
# TODO: Validate the correctness of the new "Inventory Transactions" fact table.
sql_inventory_transactions = "SELECT * FROM northwind_dw2.fact_inventory_transactions;"
df_fact_inventory_transaction = get_sql_dataframe(sql_inventory_transactions, **mysql_args)
df_fact_inventory_transaction.head(2)

,fact_inventory_transaction_key,inventory_transaction_id,inventory_transaction_type,product_key,created_date_key,modified_date_key,quantity
0,1,35,Purchased,27,20060322,20060322,75
1,2,36,Purchased,24,20060322,20060322,40


### 3.0. Demonstrate that the New Data Warehouse Exists and Contains the Correct Data
To demonstrate the viability of your solution, author a SQL SELECT statement that returns:
- Each Company’s Name
- The total amount of the purchase order detail quantity associated with each company
- The total amount of the purchase order detail unit cost associated with each company

**NOTE:** *Remember that a string typed variable whose value is contained by triple-quotes (""" ... """) can preserve multi-line formatting, and that a string variable has an intrinsic **.format()** function that accepts ordered parameters that will replace tokens (e.g., {0}) in the formatted string.*  

In [118]:
sql_purchase_orders = """
SELECT
    s.company AS Company,
    SUM(COALESCE(po.po_detail_quantity, 0))  AS total_quantity,
    SUM(COALESCE(po.po_detail_unit_cost, 0)) AS total_unit_cost
FROM northwind_dw2.fact_purchase_orders AS po
JOIN northwind_dw2.dim_suppliers      AS s
  ON s.suppliers_key = po.supplier_key   -- << plural on the dim side, singular on the fact side
GROUP BY s.company
ORDER BY total_unit_cost DESC;
"""


In [119]:
df_fact_purchase_orders = get_sql_dataframe(sql_purchase_orders, **mysql_args)
df_fact_purchase_orders

,Company,total_quantity,total_unit_cost
0,Supplier B,1180.0,364.0
1,Supplier A,1865.0,208.0
2,Supplier C,390.0,93.0
3,Supplier E,400.0,63.0
4,Supplier F,90.0,52.0
5,Supplier D,75.0,3.0


### 3.1 Extra Credit: Author a Query that Returns the Inventory Transaction Data
- Each Product Category
- Each TransactionType
- The total number of transactions associated with each Product Category and Transaction Type
- The total amount (quantity) associated with each Product Category and Transaction Type

In [122]:
sql_inventory_transactions = """
    SELECT p.category AS 'Product Category'
    , it.inventory_transaction_type AS `Transaction Type`
    , COUNT(it.inventory_transaction_id) `Number of Transactions`
    , FORMAT(SUM(it.quantity),0) AS `Total Quantity`
FROM northwind_dw2.fact_inventory_transactions it
INNER JOIN northwind_dw2.dim_products AS p
ON p.product_key = it.product_key
GROUP BY p.category
    , it.inventory_transaction_type
ORDER BY `Number of Transactions` DESC
    , `Total Quantity` DESC;
    

"""

In [123]:
df_fact_inventory_transactions = get_sql_dataframe(sql_inventory_transactions, **mysql_args)
df_fact_inventory_transactions

,Product Category,Transaction Type,Number of Transactions,Total Quantity
0,Beverages,Purchased,10,"1,525"
1,Beverages,On Hold,8,448
2,Dried Fruit & Nuts,Sold,8,155
3,Beverages,Sold,8,"1,027"
4,Dried Fruit & Nuts,Purchased,5,215
5,Candy,Sold,5,200
6,Baked Goods & Mixes,Purchased,5,105
7,Baked Goods & Mixes,Sold,5,105
8,Sauces,Sold,4,65
9,Soups,Sold,4,290
